Install required packages

In [ ]:
! pip install -qU google-cloud-aiplatform langchain chromadb pandas

Restart the Runtime ! Runtime crashes so restart using the menu!

In [ ]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}

Authenticate the user

In [ ]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
PROJECT_ID = "ibm-keras"
REGION = "us-central1"
import vertexai
import pandas as pd
from google.cloud import aiplatform
from langchain.llms import VertexAI
from langchain.embeddings import VertexAIEmbeddings
vertexai.init(project=PROJECT_ID, location=REGION)

In [ ]:
# LLM model
llm = VertexAI(
    model_name="text-bison@001",
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Embedding
embeddings = VertexAIEmbeddings(model_name="textembedding-gecko@001")

In [ ]:
from langchain.agents import create_pandas_dataframe_agent

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/generative-ai/test.csv', sep=';')
print(df.head())

   age          job  marital  education default  balance housing loan  \
0   30   unemployed  married    primary      no     1787      no   no   
1   33     services  married  secondary      no     4789     yes  yes   
2   35   management   single   tertiary      no     1350     yes   no   
3   30   management  married   tertiary      no     1476     yes  yes   
4   59  blue-collar  married  secondary      no        0     yes   no   

    contact  day month  duration  campaign  pdays  previous poutcome   y  
0  cellular   19   oct        79         1     -1         0  unknown  no  
1  cellular   11   may       220         1    339         4  failure  no  
2  cellular   16   apr       185         1    330         1  failure  no  
3   unknown    3   jun       199         4     -1         0  unknown  no  
4   unknown    5   may       226         1     -1         0  unknown  no  


In [ ]:
agent = create_pandas_dataframe_agent(llm, df, verbose=True)

In [ ]:
agent.run("how many rows are there?")

Thought: I can use the shape attribute to get the number of rows and columns
Action: python_repl_ast
Action Input: df.shape
Observation: (4521, 17)
Thought:I now know the final answer
Final Answer: There are 4521 rows.

> Finished chain.


'There are 4521 rows.'

In [ ]:
agent.run("how many customers with age>30 in the dataset?")

Thought: I need to count the number of rows where age is greater than 30
Action: python_repl_ast
Action Input: len(df[df['age'] > 30])
Observation: 3889
Thought:I now know the final answer
Final Answer: 3889

> Finished chain.


'3889'

In [ ]:
agent.run("how many unemployed customers in the dataset?")

Thought: I need to count the number of unemployed customers in the dataset.
Action: python_repl_ast
Action Input: df['job'].value_counts()
Observation: job
management       969
blue-collar      946
technician       768
admin.           478
services         417
retired          230
self-employed    183
entrepreneur     168
unemployed       128
housemaid        112
student           84
unknown           38
Name: count, dtype: int64
Thought:I now know the final answer
Final Answer: There are 128 unemployed customers in the dataset.

> Finished chain.


'There are 128 unemployed customers in the dataset.'